In [1]:
from typing import List
from typing import Tuple
from typing import Callable
from collections import Counter
from math import sqrt
from math import pi
from math import exp
from math import erf

vector = List[float]
Matrix = List[vector]
SQRT_TWO_PI = sqrt(2 * pi)

# Linear Algebra

In [2]:
# add two vectors
def add(x: vector, y: vector) -> vector:
    assert len(x) == len(y)
    return [x_i + y_i for x_i, y_i in zip(x, y)]

# subtract two vectors
def subtract(x: vector, y: vector) -> vector:
    assert len(x) == len(y)
    return [x_i - y_i for x_i, y_i in zip(x, y)]

# component wise sum of vectors
def vector_sum(vectors: List[vector]) -> vector: 
    assert vectors, "No vectors."
    
    num_elements = len(vectors[0])
    assert all(len(vector) == num_elements for vector in vectors), "Different sizes."
    
    return [sum(vector[i] for vector in vectors) for i in range(num_elements)]

# scalar-vector multiplication
def scalar_multiply(c: float, x: vector) -> vector: 
    return [c * x_i for x_i in x]

# elementwise mean of vector
def vector_mean(vectors: List[vector]) -> vector: 
    n = len(vectors)
    
    return scalar_multiply(1/n, vector_sum(vectors))

# dot product of two vectors
def dot(x: vector, y: vector) -> float: 
    assert len(x) == len(y)
    
    return sum(x_i * y_i for x_i, y_i in zip(x, y))

# sum of squares of a vector's elements
def sum_of_squares(x: vector) -> float: 
    return dot(x, x)

# magnitude of vector
def magnitude(x: vector) -> float:
    return sqrt(sum_of_squares(x))

# distance between two vectors
def distance(x: vector, y: vector) -> float: 
    return magnitude(subtract(x, y))

# Matrix's shape
def shape(A: Matrix) -> Tuple[int, int]: 
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols

# Get matrix's row
def get_row(A: Matrix, i: int) -> vector:
    return A[i]
    
# Get matrix's column
def get_col(A: Matrix, j: int) -> vector:
    return [A_i[j] for A_i in A]

# Create a matrix
def make_matrix(num_rows: int, num_cols: int, entry_fn: Callable[[int, int], float]) -> Matrix:
    return [[entry_fn(i, j) for j in range(num_cols)] for i in range(num_rows)]

# Entry 0 value function for make_matrix
def zero_entries(i: int, j: int) -> float:
    return 0

# Identity matrix
def identity_matrix(n: int) -> Matrix:
    return make_matrix(n, n, lambda i, j: 1 if i == j else 0)

# Statistics

In [3]:
# Mean of vector
def mean(x: vector) -> float: 
    return sum(x)/len(x)

# Median of even length vector
def _median_even(x: vector) -> float:
    sorted_x = sorted(x)
    high_midpoint = len(x) // 2
    return (sorted_x[high_midpoint-1] + sorted_x[high_midpoint]) / 2

# Median of odd length vector
def _median_odd(x: vector) -> float:
    return sorted(x)[len(x) // 2]

# Median
def median(x: vector) -> float:
    return _median_even(x) if len(x) % 2 == 0 else _median_odd(x)

# Quantile
def quantile(x: vector, p: float) -> float: 
    p_index = int(p * len(x))
    return sorted(x)[p_index]

# Mode
def mode(x: vector) -> vector: 
    counts = Counter(x)
    max_count = max(counts.values())
    return [x_i for x_i, count in counts.items() if count == max_count]

# Range
def data_range(x: vector) -> float: 
    return max(x) - min(x)

# Subtracting average from each elem
def de_mean(x: vector) -> vector:
    avg = mean(x)
    return [elem - avg for elem in x]

# Variance
def variance(x: vector) -> float: 
    assert len(x) >= 2, "variance requires at least two elements."
    
    n = len(x)
    deviations = de_mean(x)
    return sum_of_squares(deviations) / (n - 1)

# Standard Deviation
def standard_deviation(x: vector) -> float: 
    return sqrt(variance(x))

# Interquartile Range
def interquartile_range(x: vector) -> float:
    return quantile(x, .75) - quantile(x, .25) 

# Covariance
def covariance(x: vector, y: vector) -> float:
    assert len(x) == len(y), "Both vectors must have same number of elements"
    
    return dot(de_mean(x), de_mean(y)) / (len(x) - 1)

# Correlation
def correlation(x: vector, y: vector) -> float: 
    stdev_x = standard_deviation(x)
    stdev_y = standard_deviation(y)
    if stdev_x > 0 and stdev_y > 0: 
        return covariance(x, y) / stdev_x / stdev_y
    else:
        return 0

# Probability

In [4]:
# Uniform Distribution Probability Density Function
def uniform_pdf(x: float) -> float:
    return 1 if 0 <= x < 1 else 0 

# Uniform Distribution Cumulative Distribution Function
def uniform_cdf(x: float) -> float: 
    if x < 0:   return 0
    elif x < 1: return x
    else:       return 1

# Normal Distribution Probability Density Function
def normal_pdf(x: float, meu: float = 0, sigma: float = 1) -> float:
    return (exp(-(x-mu) ** 2 / 2 / sigma ** 2) / (SQRT_TWO_PI * sigma))

# Normal Distribution Cumulative Distribution Function
def normal_cdf(x: float, meu: float = 0, sigma: float = 1) -> float:
    return (1 + erf((x-mu) / sqrt(2) / sigma)) / 2

# Normal Distribution CDF Inverse using Binary Search
def inverse_normal_cdf(p: float, mu: float = 0, sigma: float = 1, tolerance: float = 0.00001) -> float:
    if mu != 0 and sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0 
    high_z = 10.0
    while high_z - low_z > tolerance: 
        mid_z = (low_z + high_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            low_z = mid_z
        else: 
            high_z = mid_z
            
    return mid_z

# Bernoulli Trial 
def bernoulli_trial(p: float) -> int: 
    return 1 if random.random() < p else 0 

# Sum of n Bernoulli(p) Trials 
def binomial(n: int, p: float) -> int:
    return sum(bernoulli_trial(p) for _ in range(n))